In [ ]:
from keras.layers import Input, Dense, LSTM, AveragePooling1D, Conv1D, Reshape, BatchNormalization, Activation, Dropout, Add,MaxPooling1D
from keras.layers import GlobalAveragePooling2D, Flatten, GlobalAveragePooling1D,GlobalMaxPooling1D,ZeroPadding1D,Concatenate

from keras.models import Sequential, Model
from keras.models import load_model

from keras.optimizers import SGD, RMSprop, Adam
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score

# Load spatial feature

In [ ]:
def load_data(x,y):
    x = np.load(x)
    y = np.load(y)
    return x,y

def split_data(x,y):
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
    return X_train, X_test, y_train, y_test

# Conv1D+LSTM network

In [ ]:
def Model_(n_classes,w,h):
    input_layer = Input(shape=(w,h))
    #padding = BatchNormalization()(input_layer)
    padding = ZeroPadding1D(padding=1)(input_layer)
    conv1 = Conv1D(filters=1024,kernel_size=3,strides=1)(padding)
    conv1 = Activation('relu')(conv1)
    conv1 = Dropout(0.2)(conv1)
    conv1 = BatchNormalization()(conv1)
    pool1 = AveragePooling1D(pool_size=2, strides = 2)(conv1)
    lstm1 = LSTM(1024, dropout=0.2, recurrent_dropout=0.2,return_sequences=True)(pool1)
    lstm1 = Dropout(0.2)(lstm1)
    lstm1 = GlobalAveragePooling1D()(lstm1)
    lstm1 = BatchNormalization()(lstm1)
    output_layer = Dense(n_classes, activation='softmax')(lstm1)
    model = Model(inputs=input_layer, outputs=output_layer)
    return model       

# Fit the model

In [ ]:
def fitModel(X_train, X_test, y_train, y_test,modelFile,epochs):
    opt = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])

    reduce_lr = ReduceLROnPlateau(monitor='val_loss',min_lr=0.001,factor=0.1,patience=2)
    
    checkpointer = ModelCheckpoint(filepath=modelFile,monitor='val_loss',save_best_only=True)
    

    model.fit(X_train, y_train, batch_size=32, epochs=epochs,
                                  validation_data=(X_test, y_test), verbose=1,
                                  callbacks=[reduce_lr,checkpointer])

# Train with Food11 dataset

In [ ]:
nClasses = 11
epochs = 10

SOR_DATASET = 'feature_food11'
SOR_LABEL = 'feature_food11'
DES_MODEL_FILE = 'model'

dataFile = SOR_DATASET+"/feature_food11_ResNet50.npy"
labelFile = SOR_LABEL+"/label_food11.npy"

modelFile = DES_MODEL_FILE+"/model_conv1d_lstm_food11.hdf5"

x,y = load_data(dataFile,labelFile)

X_train, X_test, y_train, y_test = split_data(x,y)

model=Model_(nClasses,x.shape[1],x.shape[2])

fitModel(X_train, X_test, y_train, y_test,modelFile,epochs)